In [ ]:
!pip install transformers==3.0.2

     |████████████████████████████████| 769 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 29.3 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
     |████████████████████████████████| 3.0 MB 48.7 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%load_ext autoreload
%autoreload 2
import sys
# May have to change this path based on local drive setup
root_folder = "/content/drive/MyDrive/fakenews-2021"
sys.path.append(root_folder)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer, RobertaConfig
import logging
logging.basicConfig(level=logging.ERROR)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import csv
from transformers import BertConfig, BertModel

In [ ]:
pd.__version__

'1.1.5'

In [ ]:
# Task 1
col_names_1 = ['TweetID', 'Class Label', 'Tweet Text']
task_1 = pd.read_csv(root_folder+'/dev-task-1.csv', names = col_names_1, quoting=csv.QUOTE_NONE)#.set_index('TweetID')
regex = r'[^\w\s.!?\-",:;#()]'
task_1['Tweet Text'] = task_1['Tweet Text'].str.replace(regex, '')
dataset = task_1

In [ ]:
# Task 2
col_names_1 = ['TweetID', 'Class Label', 'Tweet Text']
col_names_2 = ['TweetID', 'Binary Flag for Suppressed cures','Binary Flag for Behaviour and Mind Control','Binary Flag for Antivax','Binary Flag for Fake virus*','Binary Flag for Intentional Pandemic','Binary Flag for Harmful Radiation/ Influence','Binary Flag for Population reduction','Binary Flag for New World Order','Binary Flag for Satanism', 'Tweet Text']
col_names_3 = ['TweetID', 'Class Label for Suppressed cures','Class Label for Behaviour and Mind Control','Class Label for Antivax','Class Label for Fake virus*','Class Label for Intentional Pandemic','Class Label for Harmful Radiation/ Influence','Class Label for Population reduction','Class Label for New World Order','Class Label for Satanism', 'Tweet Text']
task_2 = pd.read_csv(root_folder+'/dev-task-2.csv', names = col_names_2, quoting=csv.QUOTE_NONE)#.set_index('TweetID')
task_3 = pd.read_csv(root_folder+'/dev-task-3.csv', names = col_names_3, quoting=csv.QUOTE_NONE)#.set_index('TweetID')

In [ ]:
dataset = task_2
len(dataset)

1054

In [ ]:
dataset.head(5)

,TweetID,Binary Flag for Suppressed cures,Binary Flag for Behaviour and Mind Control,Binary Flag for Antivax,Binary Flag for Fake virus*,Binary Flag for Intentional Pandemic,Binary Flag for Harmful Radiation/ Influence,Binary Flag for Population reduction,Binary Flag for New World Order,Binary Flag for Satanism,Tweet Text
0,501,0,0,0,0,1,0,0,0,0,GLOBALISTNEWS you know that the coronavirus is...
1,502,0,0,0,0,1,0,0,1,0,DO NOT LISTEN TO World Health Organization! Th...
2,503,0,0,0,0,1,0,1,0,0,There are reports that China has over 900000 p...
3,504,0,0,0,0,1,0,1,0,0,There are reports that China has over 90000 pp...
4,505,1,0,0,0,0,0,0,0,0,"""MMS (which is mainly bleach) is also being us..."


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-5
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# 改这个（可以直接换掉这个class）
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['Tweet Text'].to_numpy()
        # task1
        if 'Class Label' in self.data.columns:
            self.targets = self.data['Class Label'].to_numpy() - 1
        # task 2
        elif 'Binary Flag for Suppressed cures' in self.data.columns:
            self.targets = self.data[col_names_2[1:-1]].to_numpy().astype(float)
        # task 3
        else:
            self.targets = self.data[col_names_3[1:-1]].to_numpy().astype(float) - 1
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_size = 0.8
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=0)
# train_data=train.sample(frac=train_size,random_state=200)
# test_data=train.drop(train_data.index).reset_index(drop=True)
# train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (1054, 11)
TRAIN Dataset: (843, 11)
TEST Dataset: (211, 11)


In [ ]:
len(train_data)

843

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        #self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.l1 = BertModel.from_pretrained("bert-base-uncased")
        #config = BertConfig()
        #self.l1 = BertModel(config)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) #set output_hidden_states=True
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]

        #output_2 = self.l2(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        #hidden_state2 = output_2[0]
        #pooler2 = hidden_state2[:, 0]
        #l10 = ..
        pooler = self.pre_classifier(pooler)
        #pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

RobertaClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [ ]:
class RobertaMultitask(torch.nn.Module): # 改这里
    def __init__(self):
        super(RobertaMultitask, self).__init__() # 改这里
        self.l1 = BertModel.from_pretrained("bert-base-uncased")
        #config = BertConfig()
        #self.l1 = BertModel(config)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2 * 9) # 改这里
        #self.classifier2 = torch.nn.Linear(768, 3)
        self.classifier3 = torch.nn.Linear(27, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) #set output_hidden_states=True
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]

        #output_2 = self.l2(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        #hidden_state2 = output_2[0]
        #pooler2 = hidden_state2[:, 0]
        #l10 = ..
        pooler = self.pre_classifier(pooler)
        #pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = output.view(-1, 2) # 改这里
        return output

    def task1(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) #set output_hidden_states=True
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]

        #output_2 = self.l2(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        #hidden_state2 = output_2[0]
        #pooler2 = hidden_state2[:, 0]
        #l10 = ..
        pooler = self.pre_classifier(pooler)
        #pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.classifier3(output)
        return output

In [ ]:
model = RobertaMultitask()
model.to(device)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

RobertaMultitask(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
class RobertaFromScratch(torch.nn.Module):
    def __init__(self):
        super(RobertaFromScratch, self).__init__()
        #self.l1 = RobertaModel.from_pretrained("roberta-base")
        #self.l1 = BertModel.from_pretrained("bert-base-uncased")
        config = BertConfig()
        self.l1 = BertModel(config)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) #set output_hidden_states=True
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        #l10 = ..
        #pooler = self.pre_classifier(torch.cat([l10, pooler]))
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
#model = RobertaFromScratch()
#model.to(device)

In [ ]:
# Creating the loss function and optimizer
#loss_function = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.05, 0.6, 0.35]).to(device))
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calculate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def calculate_mcc(preds, targets):
    return sklearn.metrics.matthews_corrcoef(targets, preds)

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _, data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        targets = targets.flatten() # 加这里

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%50==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 50 steps: {loss_step}")
            print(f"Training Accuracy per 50 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train(epoch)
    acc = valid(model, testing_loader)
    print("Accuracy on test data = %0.2f%%" % acc)

0it [00:00, ?it/s]

Training Loss per 50 steps: 0.30795520544052124
Training Accuracy per 50 steps: 88.88888888888889


50it [00:26,  1.86it/s]

Training Loss per 50 steps: 0.2825852590156536
Training Accuracy per 50 steps: 89.92374727668846


100it [00:53,  1.86it/s]

...

The Total Accuracy for Epoch 4: 97.25846843284566
Training Loss Epoch: 0.09970989049151893
Training Accuracy Epoch: 97.25846843284566


1it [00:00,  5.39it/s]

Validation Loss per 100 steps: 0.03883242607116699
Validation Accuracy per 100 steps: 100.0


53it [00:09,  5.71it/s]

Validation Loss Epoch: 0.2048632583387618
Validation Accuracy Epoch: 92.78567667193259
Validation MCC Epoch: 0.5798269683794527
Accuracy on test data = 92.79%


In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)
    acc = valid(model, testing_loader)
    print("Accuracy on test data = %0.2f%%" % acc)

0it [00:00, ?it/s]

Training Loss per 50 steps: 0.08158054947853088
Training Accuracy per 50 steps: 97.22222222222223


50it [00:26,  1.85it/s]

Training Loss per 50 steps: 0.07635061344241395
Training Accuracy per 50 steps: 98.20261437908496


100it [00:53,  1.85it/s]

...

The Total Accuracy for Epoch 0: 98.23382100962172
Training Loss Epoch: 0.07208511084582991
Training Accuracy Epoch: 98.23382100962172


1it [00:00,  5.51it/s]

Validation Loss per 100 steps: 0.287019282579422
Validation Accuracy per 100 steps: 91.66666666666667


53it [00:09,  5.71it/s]

Validation Loss Epoch: 0.21112279984045704
Validation Accuracy Epoch: 92.25908372827804
Validation MCC Epoch: 0.548004296928206
Accuracy on test data = 92.26%


In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 50 steps: 0.2361990064382553
Training Accuracy per 50 steps: 94.44444444444444


50it [00:21,  2.35it/s]

Training Loss per 50 steps: 0.3310772668497235
Training Accuracy per 50 steps: 89.27015250544662


100it [00:42,  2.35it/s]

...

Training Loss per 50 steps: 0.3228324679349432
Training Accuracy per 50 steps: 89.25680647534952


200it [01:25,  2.36it/s]

Training Loss per 50 steps: 0.3156404518591824
Training Accuracy per 50 steps: 89.37258153676063


208it [01:28,  2.35it/s]


KeyboardInterrupt: ignored

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    outputs_lst, targets_lst = [], []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            targets = targets.flatten()
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            #print(big_idx, targets)
            n_correct += calculate_accuracy(big_idx, targets)
            outputs_lst.extend(list(big_idx.cpu()))
            targets_lst.extend(list(targets.cpu()))
            #print(big_idx)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    print(f"Validation MCC Epoch: {calculate_mcc(outputs_lst, targets_lst)}")
    
    return epoch_accu


In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

2it [00:00,  6.04it/s]

Validation Loss per 100 steps: 0.6752351522445679
Validation Accuracy per 100 steps: 97.22222222222223


53it [00:07,  7.27it/s]

Validation Loss Epoch: 0.6867196869175389
Validation Accuracy Epoch: 88.78357030015798
Validation MCC Epoch: 0.5080344525376046
Accuracy on test data = 88.78%


In [ ]:
class SentimentData2(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['Tweet Text'].to_numpy()
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

In [ ]:
dataset2 = pd.read_csv(root_folder+'/test1.csv', quoting=csv.QUOTE_NONE).astype({'Tweet Text': str})
testing_set2 = SentimentData2(dataset2, tokenizer, MAX_LEN)
testing_loader2 = DataLoader(testing_set2, **test_params)

In [ ]:
def test(model, testing_loader2):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    outputs_lst = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader2, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze().cpu()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            outputs_lst.extend(list(big_idx))
    print(outputs_lst)
    return outputs_lst

outputs = test(model,testing_loader2)

67it [00:09,  7.35it/s]

[tensor(0), tensor(0), tensor(1), tensor(2), tensor(2), tensor(2), tensor(2), tensor(2), tensor(1), tensor(0), tensor(0), tensor(2), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(2), tensor(1), tensor(2), tensor(1), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(2), tensor(0), tensor(1), tensor(0), tensor(2), tensor(0), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(2), tensor(1), tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(2), tensor(2), tensor(0), tensor(2), tensor(0), tensor(0), tensor(2), tensor(1), tensor(0), tensor(2), tensor(2), tensor(1), tensor(0), tensor(0), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(2), tensor(2), tensor(0), tensor(0), tensor(2), tensor(2), tensor(2), tensor(2), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(2), tensor(2), tensor(0)

In [ ]:
rows = []
index = 2084
print(int(outputs[0]))
headers = ['TweetID', 'Class Label for Suppressed cures','Class Label for Behaviour and Mind Control','Class Label for Antivax','Class Label for Fake virus*','Class Label for Intentional Pandemic','Class Label for Harmful Radiation/ Influence','Class Label for Population reduction','Class Label for New World Order','Class Label for Satanism']
for i in range(0,len(outputs)):
  #rows.append([index,int(outputs[i])+1,int(outputs[i+1])+1,int(outputs[i+2])+1,int(outputs[i+3])+1,int(outputs[i+4])+1,int(outputs[i+5])+1,int(outputs[i+6])+1,int(outputs[i+7])+1,int(outputs[i+8])+1])
  #rows.append([index,int(outputs[i]),int(outputs[i+1]),int(outputs[i+2]),int(outputs[i+3]),int(outputs[i+4]),int(outputs[i+5]),int(outputs[i+6]),int(outputs[i+7]),int(outputs[i+8])])
  rows.append([index, int(outputs[i].item())])
  index += 1
  
with open('result1.csv','w') as f:
    f_csv = csv.writer(f)
    #f_csv.writerow(headers)
    f_csv.writerows(rows)

0


In [ ]:
l12 * a + l10 * b = 768
concat([l12, l10]) -> 768*2

In [ ]:
dataset = task_1
train_size = 0.8
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=0)
# train_data=train.sample(frac=train_size,random_state=200)
# test_data=train.drop(train_data.index).reset_index(drop=True)
# train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

FULL Dataset: (1054, 3)
TRAIN Dataset: (843, 3)
TEST Dataset: (211, 3)


In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _, data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        targets = targets.flatten() # 加这里

        outputs = model.task1(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%50==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 50 steps: {loss_step}")
            print(f"Training Accuracy per 50 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Training Loss per 50 steps: 1.2283296585083008
Training Accuracy per 50 steps: 0.0


50it [00:21,  2.35it/s]

Training Loss per 50 steps: 1.1706554644248064
Training Accuracy per 50 steps: 39.705882352941174


100it [00:42,  2.37it/s]

...

Training Loss per 50 steps: 1.0336705513347852
Training Accuracy per 50 steps: 44.03973509933775


200it [01:24,  2.36it/s]

Training Loss per 50 steps: 1.0402006076936106
Training Accuracy per 50 steps: 43.407960199004975


211it [01:29,  2.37it/s]

The Total Accuracy for Epoch 2: 43.890865954922894
Training Loss Epoch: 1.0391963683598415
Training Accuracy Epoch: 43.890865954922894
